In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import datetime
import matplotlib.pyplot as plt
import matplotlib
import re
from scipy import stats

matplotlib.rcParams['figure.figsize'] = (10, 5)
matplotlib.rcParams['font.size'] = 12

import random
random.seed(1)
import time

import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import get_scorer
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
import lightgbm as lgb
from sklearn.externals.joblib import Parallel, delayed
from sklearn.base import clone

import pickle

# 1.获取数据

创建环境

In [ ]:
from kaggle.competitions import twosigmanews
# You can only call make_env() once, so don't lose it!
env = twosigmanews.make_env()
print('Done!')

In [ ]:
(market_train_origin, news_train_origin) = env.get_training_data()

# 2.数据探索

# 3.数据预处理
- 排序：按时间排序
- 复制：复制数据
- 删除：删除变量，减少内存消耗

In [ ]:
market_train_origin = market_train_origin.sort_values('time')
news_train_origin = news_train_origin.sort_values('time')
market_train_df = market_train_origin.copy()
news_train_df = news_train_origin.copy()
del market_train_origin
del news_train_origin

删除经济危机之前的数据，因为经济危机对于全球股市产生了动荡的影响

In [ ]:
market_train_df = market_train_df.loc[market_train_df['time'].dt.date>=datetime.date(2009,1,1)]
news_train_df = news_train_df.loc[news_train_df['time'].dt.date>=datetime.date(2009,1,1)]

## 市场数据处理
- 去除离群点 （Open to close）：开盘价格和收盘价格不应该太大（太大的话，市场就崩溃了）。在这里，我们定义一个收盘/开盘比率，超过或者低于这个比率的定义为离群点，然后删除整行数据。
>- 收盘/开盘 >= 1.5 - 股票增长超过50%
>- 收盘/开盘 <=0.5 - 股票减少超过50%
>- 0.5 <收盘/开盘<1.5 - 正常的股票波动


In [ ]:
market_train_df['close_open_ratio'] = np.abs(market_train_df['close']/market_train_df['open'])
threshold = 0.5
print('In %i lines price increases by 50%% or more in a day' %(market_train_df['close_open_ratio']>=1.5).sum())
print('In %i lines price decreases by 50%% or more in a day' %(market_train_df['close_open_ratio']<=0.5).sum())

In [ ]:
market_train_df = market_train_df.loc[market_train_df['close_open_ratio'] < 1.5]
market_train_df = market_train_df.loc[market_train_df['close_open_ratio'] > 0.5]
market_train_df = market_train_df.drop(columns=['close_open_ratio'])

- 填充零值点
> 零值点都在market adjusted 中，所以我们用raw值来填充

In [ ]:
column_market = ['returnsClosePrevMktres1','returnsOpenPrevMktres1','returnsClosePrevMktres10', 'returnsOpenPrevMktres10']
column_raw = ['returnsClosePrevRaw1', 'returnsOpenPrevRaw1','returnsClosePrevRaw10', 'returnsOpenPrevRaw10']
for i in range(len(column_raw)):
    market_train_df[column_market[i]] = market_train_df[column_market[i]].fillna(market_train_df[column_raw[i]])

- 去除零值点（Returns）
>returns收益不可能增加或减少超过50%，不然就是噪声或者极值，这些值需要被移除

In [ ]:
print('Removing outliers ...')
column_return = column_market + column_raw + ['returnsOpenNextMktres10']
orig_len = market_train_df.shape[0]
for column in column_return:
    market_train_df = market_train_df.loc[market_train_df[column]>=-2]
    market_train_df = market_train_df.loc[market_train_df[column]<=2]
new_len = market_train_df.shape[0]
rmv_len = np.abs(orig_len-new_len)
print('There were %i lines removed' %rmv_len)

- 移除陌生值：asset和 asset code 如果有 ‘unknown’ 需要被移除，它们的市场表现比较特殊，需要移除'PGN.N','EBRYY.OB'的原因具体参考这里: https://www.kaggle.com/nareyko/market-return-estimation-and-bad-data-detection

In [ ]:
print('Removing strange data ...')
orig_len = market_train_df.shape[0]
market_train_df = market_train_df[~market_train_df['assetCode'].isin(['PGN.N','EBRYY.OB'])]
#market_train_df = market_train_df[~market_train_df['assetName'].isin(['Unknown'])]
new_len = market_train_df.shape[0]
rmv_len = np.abs(orig_len-new_len)
print('There were %i lines removed' %rmv_len)

## 新闻数据处理
- 处理离群点
>取分为点位于0.02到0.98的数据

In [ ]:
# Function to remove outliers
def remove_outliers(data_frame, column_list, low=0.02, high=0.98):
    for column in column_list:
        this_column = data_frame[column]
        quant_df = this_column.quantile([low,high])
        low_limit = quant_df[low]
        high_limit = quant_df[high]
        data_frame[column] = data_frame[column].clip(lower=low_limit, upper=high_limit)
    return data_frame


In [ ]:
# Remove outlier
columns_outlier = ['takeSequence', 'bodySize', 'sentenceCount', 'wordCount', 'sentimentWordCount', 'firstMentionSentence','noveltyCount12H',\
                  'noveltyCount24H', 'noveltyCount3D', 'noveltyCount5D', 'noveltyCount7D', 'volumeCounts12H', 'volumeCounts24H',\
                  'volumeCounts3D','volumeCounts5D','volumeCounts7D']
print('Clipping news outliers ...')
news_train_df = remove_outliers(news_train_df, columns_outlier)

# 4.特征工程

- 合成function
>把市场和新闻信息合并在一起
- drop_colums
>只取有意义的colums

In [ ]:
asset_code_dict = {k: v for v, k in enumerate(market_train_df['assetCode'].unique())}
drop_columns = [col for col in news_train_df.columns if col not in ['sourceTimestamp', 'urgency', 'takeSequence', 'bodySize', 'companyCount', 
               'sentenceCount', 'firstMentionSentence', 'relevance','firstCreated', 'assetCodes']]
columns_news = ['firstCreated','relevance','sentimentClass','sentimentNegative','sentimentNeutral',
               'sentimentPositive','noveltyCount24H','noveltyCount7D','volumeCounts24H','volumeCounts7D','assetCodes','sourceTimestamp',
               'assetName','audiences', 'urgency', 'takeSequence', 'bodySize', 'companyCount', 
               'sentenceCount', 'firstMentionSentence','time']

In [ ]:
# Data processing function
def data_prep(market_df,news_df):
    market_df['date'] = market_df.time.dt.date#转换时间格式
    market_df['close_to_open'] = market_df['close'] / market_df['open']#创建一个新的colum，写入close/open比率
    market_df.drop(['time'], axis=1, inplace=True)#drop到colum time
    
    news_df = news_df[columns_news]#只取定义的feature（colums）
    news_df['sourceTimestamp']= news_df.sourceTimestamp.dt.hour#转换成小时
    news_df['firstCreated'] = news_df.firstCreated.dt.date#转换日期格式
    news_df['assetCodesLen'] = news_df['assetCodes'].map(lambda x: len(eval(x)))#计算assetcode的数量然后创建一个新的colum
    news_df['assetCodes'] = news_df['assetCodes'].map(lambda x: list(eval(x))[0])#取第一个assetcode并替换
    news_df['asset_sentiment_count'] = news_df.groupby(['assetName', 'sentimentClass'])['time'].transform('count')# 按照assetname和sentimentclas分组，并统计assetName和sentimentclass为一对的数量，然后创建column
    news_df['len_audiences'] = news_train_df['audiences'].map(lambda x: len(eval(x)))
    kcol = ['firstCreated', 'assetCodes']
    news_df = news_df.groupby(kcol, as_index=False).mean()#按照firstcreated 和 assetcodes分组，并计算个column平均值
    market_df = pd.merge(market_df, news_df, how='left', left_on=['date', 'assetCode'], 
                            right_on=['firstCreated', 'assetCodes'])#按照left（market），key是date 和 assetcode合并
    del news_df
    market_df['assetCodeT'] = market_df['assetCode'].map(asset_code_dict)#把assetcode map到assetcodeT 中index
    market_df = market_df.drop(columns = ['firstCreated','assetCodes','assetName']).fillna(0) #drop 3个columns 然后把nan转成0
    return market_df

In [ ]:
print('Merging data ...')
market_train_df = data_prep(market_train_df, news_train_df)
market_train_df.head()

## 数据选择
- 2009年经济危机之后的数据

In [ ]:
market_train_df = market_train_df.loc[market_train_df['date']>=datetime.date(2009,1,1)]

num_columns - 数值特征
cat_columns - 类别特征

In [ ]:
num_columns = ['volume', 'close', 'open', 'returnsClosePrevRaw1', 'returnsOpenPrevRaw1', 'returnsClosePrevMktres1', 'returnsOpenPrevMktres1', 'returnsClosePrevRaw10', 'returnsOpenPrevRaw10', 
               'returnsClosePrevMktres10', 'returnsOpenPrevMktres10', 'close_to_open', 'sourceTimestamp', 'urgency', 'companyCount', 'takeSequence', 'bodySize', 'sentenceCount',
               'relevance', 'sentimentClass', 'sentimentNegative', 'sentimentNeutral', 'sentimentPositive',
               'noveltyCount24H','noveltyCount7D','volumeCounts24H','volumeCounts7D','assetCodesLen', 'asset_sentiment_count', 'len_audiences']
cat_columns = ['assetCodeT']
feature_columns = num_columns+cat_columns

由于使用tree-based model 所以不用feature standarlization，只要Normalization就行

| Normalization   | Standardization |
| ------------- | ------------- |
| Normalization 会使得所有值缩放为（0,1）范围 | Standardization 范围不确定，但是均值为0方差为1  |
| 如有异常值，Normalization不利于建模  | 范围不确定，对于某些算法不利 |

In [ ]:
# Scaling of data
#from sklearn.preprocessing import StandardScaler, MinMaxScaler

#data_scaler = StandardScaler()
#market_train_df[num_columns] = data_scaler.fit_transform(market_train_df[num_columns])
#data_scaler = MinMaxScaler()
#market_train_df[num_columns] = data_scaler.fit_transform(market_train_df[num_columns])

In [ ]:
from sklearn.model_selection import train_test_split

market_train_df = market_train_df.reset_index()
market_train_df = market_train_df.drop(columns='index')

# Random train-test split
train_indices, val_indices = train_test_split(market_train_df.index.values,test_size=0.1, random_state=92)

转化为分类问题, 0为回报（returns），1为正回报（returns）
- returnsOpenNextMktres10<0 => 0
- returnsOpenNextMktres10>0 => 1

In [ ]:
# Extract X and Y
def get_input(market_train, indices):
    X = market_train.loc[indices, feature_columns].values
    y = market_train.loc[indices,'returnsOpenNextMktres10'].map(lambda x: 0 if x<0 else 1).values
    #y = market_train.loc[indices,'returnsOpenNextMktres10'].map(lambda x: convert_to_class(x)).values
    r = market_train.loc[indices,'returnsOpenNextMktres10'].values
    u = market_train.loc[indices, 'universe']
    d = market_train.loc[indices, 'date']
    return X,y,r,u,d

# r, u and d are used to calculate the scoring metric
X_train,y_train,r_train,u_train,d_train = get_input(market_train_df, train_indices)
X_val,y_val,r_val,u_val,d_val = get_input(market_train_df, val_indices)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca_train = PCA(whiten=True,random_state=80,svd_solver='randomized')

In [ ]:
pca_train.fit(X_train)
display(pca_train.components_.shape)
display(pca_train.components_)
display(pca_train.explained_variance_ratio_.shape)

In [ ]:
display(pca_train.explained_variance_ratio_.r

# 5.模型

- fit参数 - https://lightgbm.readthedocs.io/en/latest/Python-API.html#lightgbm.LGBMClassifier.fit

- 定义learning_rate_power为学习率的衰减方式

In [ ]:
# Set up decay learning rate
def learning_rate_power(current_round):
    base_learning_rate = 0.19000424246380565
    min_learning_rate = 0.01
    lr = base_learning_rate * np.power(0.995,current_round)
    return max(lr, min_learning_rate)

In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

#调优参数
tune_params = {'n_estimators': [200,500,1000],#[200,500,1000,2500,5000]
              'max_depth': sp_randint(4,12),
              'colsample_bytree':sp_uniform(loc=0.8, scale=0.15),
              'min_child_samples':sp_randint(60,120),
              'subsample': sp_uniform(loc=0.75, scale=0.25),#采样率uniform分布
              'reg_lambda':[1e-3, 1e-2, 1e-1, 1]}
#训练参数
fit_params = {'early_stopping_rounds':40,#如果40轮不提高，停止
              'eval_metric': 'accuracy',
              'eval_set': [(X_train, y_train), (X_val, y_val)],
              'verbose': 20,#每20打印结果
              'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_power)]}

**调优**

- LGBMClassifier - https://lightgbm.readthedocs.io/en/latest/Python-API.html#lightgbm.LGBMClassifier

- RandomsearchCV - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV

**建议单独运行RandomizedSearchCV**，Cv耗时超过3个小时

In [ ]:
lgb_clf = lgb.LGBMClassifier(n_jobs=4, objective='binary',random_state=1)
gs = RandomizedSearchCV(estimator=lgb_clf, 
                        param_distributions=tune_params, 
                        n_iter=40,
                        scoring='f1',
                        cv=5,
                        refit=True,
                        random_state=1,
                        verbose=True)
#gs.fit(X_train, y_train,**fit_params)
#best_clf = gs.best_estimator_ 
#print ("best_clf\n------")
#print (best_clf)


**模型选择**

- 根据best_clf调整opt_params

In [ ]:

lgb_clf = lgb.LGBMClassifier(n_jobs=4,
                             objective='binary',
                            random_state=100)

opt_params = {'n_estimators':500,
              'boosting_type': 'dart',
              'objective': 'binary',
              'num_leaves':2452,
              'min_child_samples':212,
              'reg_lambda':0.01}

lgb_clf.set_params(**opt_params)
lgb_clf.fit(X_train, y_train,**fit_params)


In [ ]:
print('Training accuracy: ', accuracy_score(y_train, lgb_clf.predict(X_train)))
print('Validation accuracy: ', accuracy_score(y_val, lgb_clf.predict(X_val)))

- 调整置信区间的范围
>按照target的值scale预测值

In [ ]:
features_imp = pd.DataFrame()
features_imp['features'] = list(feature_columns)[:]
features_imp['importance'] = lgb_clf.feature_importances_
features_imp = features_imp.sort_values(by='importance', ascending=False).reset_index()

y_plot = -np.arange(15)
plt.figure(figsize=(10,6))
plt.barh(y_plot, features_imp.loc[:14,'importance'].values)
plt.yticks(y_plot,(features_imp.loc[:14,'features']))
plt.xlabel('Feature importance')
plt.title('Features importance')
plt.tight_layout()

In [ ]:
# Rescale confidence
def rescale(data_in, data_ref):
    scaler_ref =  StandardScaler()
    scaler_ref.fit(data_ref.reshape(-1,1))#变成一列
    scaler_in = StandardScaler()
    data_in = scaler_in.fit_transform(data_in.reshape(-1,1))
    data_in = scaler_ref.inverse_transform(data_in)[:,0]#第一列
    return data_in

In [ ]:
y_pred_proba = lgb_clf.predict_proba(X_val)
predicted_return = y_pred_proba[:,1] - y_pred_proba[:,0]#计算return 用close - open
predicted_return = rescale(predicted_return, r_train)

In [ ]:
# calculation of actual metric that is used to calculate final score
r_val = r_val.clip(-1,1) # 大于1，小于0的去除
x_t_i = predicted_return * r_val * u_val#universe*回报*universe(0 or 1, 0这天不计入)
data = {'day' : d_val, 'x_t_i' : x_t_i}
df = pd.DataFrame(data)
x_t = df.groupby('day').sum().values.flatten()
mean = np.mean(x_t)
std = np.std(x_t)
score_valid = mean / std
print('Validation score', score_valid)

# 6.提交

In [ ]:
days = env.get_prediction_days()
n_days = 0
prep_time = 0
prediction_time = 0
packaging_time = 0
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    n_days +=1
    if n_days % 50 == 0:
        print(n_days,end=' ')

    t = time.time()
    column_market = ['returnsClosePrevMktres1','returnsOpenPrevMktres1','returnsClosePrevMktres10', 'returnsOpenPrevMktres10']
    column_raw = ['returnsClosePrevRaw1', 'returnsOpenPrevRaw1','returnsClosePrevRaw10', 'returnsOpenPrevRaw10']
    market_obs_df['close_open_ratio'] = np.abs(market_obs_df['close']/market_obs_df['open'])
    for i in range(len(column_raw)):
        market_obs_df[column_market[i]] = market_obs_df[column_market[i]].fillna(market_obs_df[column_raw[i]])

    market_obs_df = market_obs_df[market_obs_df.assetCode.isin(predictions_template_df.assetCode)]
    market_obs_df = market_obs_df[market_obs_df.assetCode.isin(asset_code_dict.keys())]
    market_obs = data_prep(market_obs_df, news_obs_df)
    market_obs[num_columns] = data_scaler.transform(market_obs[num_columns])
    X_live = market_obs[feature_columns].values
    prep_time += time.time() - t

    t = time.time()
    lp = lgb_clf.predict_proba(X_live)
    prediction_time += time.time() -t

    t = time.time()
    confidence = lp[:,1] - lp[:,0]
    #confidence = confidence_out(lp)
    confidence = rescale(confidence, r_train)
    preds = pd.DataFrame({'assetCode':market_obs['assetCode'],'confidence':confidence})
    predictions_template_df = predictions_template_df.merge(preds,how='left').drop('confidenceValue',axis=1).fillna(0).rename(columns={'confidence':'confidenceValue'})
    env.predict(predictions_template_df)
    packaging_time += time.time() - t

env.write_submission_file()